### NYC Rent Stabilized Buildings List (from DHCR)
2016 Building Registrations filed with NYS Division of Housing and Community Renewal (DHCR)

http://www1.nyc.gov/site/rentguidelinesboard/resources/rent-stabilized-building-lists.page


In [1]:
#download pdfs from website

import urllib2

borolist= ['manhattan','bronx','brooklyn','queens','statenisl']

for boro in borolist:
    url='http://www1.nyc.gov/assets/rentguidelinesboard/pdf/2016'+boro+'bldgs.pdf'
    filedata = urllib2.urlopen(url)  
    datatowrite = filedata.read()
    
    with open('dhcr_'+boro+'.pdf', 'wb') as f:  
        f.write(datatowrite)


In [2]:
# scrape pdf tables into a dataframe
import pandas as pd
from tabula import read_pdf
import pyPdf

dhcr = pd.DataFrame()
for boro in borolist:
    filename= 'dhcr_'+boro+'.pdf'
    n=pyPdf.PdfFileReader(open(filename)).getNumPages()
    df=read_pdf(filename,pages=range(1,n+1))
    df['Borough']=borolist.index(boro)+1
    dhcr=pd.concat([dhcr,df], ignore_index=True)
    
dhcr['Borough'] = pd.to_numeric(dhcr.Borough,errors='coerse')
dhcr['BLOCK'] = pd.to_numeric(dhcr.BLOCK,errors='coerse')
dhcr['LOT'] = pd.to_numeric(dhcr.LOT,errors='coerse')

dhcr['BBL'] = dhcr.Borough.apply(lambda x: '{:01.0f}'.format(x)).astype(str) +\
dhcr.BLOCK.apply(lambda x: '{:05.0f}'.format(x)).astype(str) +\
dhcr.LOT.apply(lambda x: '{:04.0f}'.format(x)).astype(str)

dhcr['BBL'] = pd.to_numeric(dhcr.BBL,errors='coerse')

dhcr.shape

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


(44444, 15)

In [3]:
dhcr.head()

,ZIP,BLDGNO1,STREET1,STSUFX1,BLDGNO2,STREET2,STSUFX2,CITY,STATUS1,STATUS2,STATUS3,BLOCK,LOT,Borough,BBL
0,10001,244,10TH,AVE,NaN,NaN,NaN,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,722.0,2.0,1,1.007220e+09
1,10001,246,10TH,AVE,NaN,NaN,NaN,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,722.0,3.0,1,1.007220e+09
2,10001,299,10TH,AVE,NaN,NaN,NaN,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,699.0,31.0,1,1.006990e+09
3,10001,301,10TH,AVE,NaN,NaN,NaN,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,699.0,32.0,1,1.006990e+09
4,10001,303 TO 309,10TH,AVE,NaN,NaN,NaN,NEW YORK,MULTIPLE DWELLING A,421-A,NaN,699.0,33.0,1,1.006990e+09


In [4]:
pd.DataFrame({'unique values':dhcr.apply(lambda x: x.nunique()),
                         'count':dhcr.apply(lambda x: x.count())})

,count,unique values
ZIP,44444,177
BLDGNO1,44444,11956
STREET1,44444,2167
STSUFX1,43704,56
BLDGNO2,2046,1005
STREET2,2046,615
STSUFX2,1993,25
CITY,44444,51
STATUS1,44443,3
STATUS2,11770,13


In [5]:
# write to csv file

dhcr.to_csv('dhcr.csv',index=False)